In [107]:
import readline
import rpy2.robjects
import code.independence_test as it
import code.additive_noise as an
import numpy as np
import numpy.random as rn
import code.hsic as hs
import pandas as pd

ImportError: cannot import name division

In [ ]:
n=200
uni = rn.uniform(0,10,n)
exp = rn.exponential(7,n)
gamma = rn.gamma(10,1,n)
log = rn.logistic(10,0.5,n)
n_Y = np.random.uniform(0,0.01,n)
n_L = np.random.uniform(0,0.01,n)
#Y = uni + exp + gamma + log + n_Y
Y = gamma + n_Y
leak = Y + n_L

In [ ]:
X = np.stack([gamma, leak, log],axis=1)

In [ ]:
X = pd.DataFrame(X, columns=['cause', 'leak', 'independent'])

In [ ]:
an.ANM_algorithm(X, Y, 'py')

In [164]:
SIZES = [200] * 2

In [172]:
# Distributions
D = {'standard_normal': lambda: rn.randn(SIZE),
     'uniform': lambda: rn.uniform(0, 1, SIZE)
}

# Function transformations
non_gaussian_noise = lambda: rn.uniform(-0.001, 0.001, SIZE)
gaussian_noise = lambda: rn.normal(0, 0.001, SIZE)

F = {
    'identity': lambda x: x + non_gaussian_noise(),
    'scale': lambda x: (rn.randint(1, 10) * rn.randn(1) * x) + non_gaussian_noise(),
    'exp': lambda x: np.exp(x.astype('float')) + gaussian_noise()
    #'square': lambda x: x ** 2,
    #'cube': lambda x: x ** 3,
}

def add(a, b):
    if a is None:
        a = np.zeros_like(a)
    return a + b

def sub(a, b):
    if a is None:
        a = np.zeros_like(a)
    return a - b

def mul(a, b):
    if a is None:
        a = np.ones_like(a)
    return a * b

def div(a, b):
    if a is None:
        a = np.ones_like(a)
    return a / b

# Possible combinations of features
C = {
    '+': add
    #'-': sub,
    #'*': mul,
    #'/': div
}

In [173]:
tests = pd.DataFrame()
tests_features = []
tests_ys = []

In [174]:
for iteration, SIZE in enumerate(SIZES):
    # Total numbr of features
    N = rn.randint(1, 5)

    # Selected to form Y
    if N > 1:
        S = rn.randint(1, N)
    else:
        S = 1

    L = 1

    print N, S, L

    # Generate Features
    all_features = {}

    for i in range(N):
        dist = rn.randint(0, len(D))
        feature = {}
        feature['dist'] = D.keys()[dist]
        feature['values'] = D.values()[dist]()
        all_features['X_{}'.format(i)] = feature

    # Aplly transformations to features
    candidates = list(range(N))
    rn.shuffle(candidates)

    for i in candidates[:S]:
        func = rn.randint(0, len(F))
        feature = all_features['X_{}'.format(i)]
        feature['func'] = F.keys()[func]
        feature['transformed'] = F.values()[func](feature['values'])

    Y = {"comb":"Y = ",
         "values": None}

    for i in range(N):
        if 'transformed' in all_features.values()[i].keys():
            comb = rn.randint(0, len(C))
            Y['comb'] += "{} f({}) ".format(C.keys()[comb], all_features.keys()[i])
            Y['values'] = C.values()[comb](Y['values'], all_features.values()[i]['transformed'])

    for i in range(L):
        feature = {}
        func = rn.randint(0, len(F))
        feature['func'] = F.keys()[func]
        feature['values'] = F.values()[func](Y['values'])
        all_features['leak_{}'.format(i)] = feature
    
    columns = [name for name in all_features.keys()]
    causes = [i for i, f in enumerate(all_features.values()) if f.get('transformed', np.array([])).shape[0] > 0]
    for j in range(len(columns)):
        if j in causes:
            columns[j] = 'cause_' + columns[j]
        elif not columns[j].startswith('leak'):
            columns[j] = 'independent_' + columns[j]

    tests_features.append(all_features)
    tests_ys.append(Y)

    final_data = np.stack([f['values'] for f in all_features.values()], axis=1).astype('float')
    data = pd.DataFrame(final_data, columns=columns)
    
    res = an.ANM_algorithm(data, Y['values'].astype('float'))
    
    res['Vars'] = res.index + '_' + str(iteration)
    res['Truth'] = [idx.split('_')[0] for idx in res.index]
    
    all_tests = all_tests.append(res, ignore_index=True)

2 1 1
Number of line searches 28
Number of line searches 27
Number of line searches 40
Number of line searches 40
Number of line searches 19
Number of line searches 27
2 1 1
Number of line searches 9
Number of line searches 11
Number of line searches 9
Number of line searches 13
Number of line searches 40
Number of line searches 40


In [175]:
all_tests

,Direction,Rank,P_value Independence X and Y,Vars,Truth
0,Leakage,2.26284e-05,2.58231e-08,cause_X_0_0,cause
1,Leakage,1.07409e-05,1.6989e-44,cause_X_1_0,cause
2,Independent,9.38215e-09,0.693891,independent_X_2_0,independent
3,No Leakage,-1.17319e-11,2.13043e-76,leak_0_0,leak
4,Leakage,1.02178e-12,2.6294e-70,cause_X_1_0,cause
5,Leakage,1.4754e-13,2.63461e-70,leak_0_0,leak
6,Independent,-1.3255e-09,0.122771,independent_X_0_0,independent
7,Leakage,7.9109e-11,3.85594e-87,cause_X_0_1,cause
8,No Leakage,-1.51846e-10,3.84869e-87,leak_0_1,leak
9,Independent,-1.76997e-05,0.185338,independent_X_1_1,independent


In [179]:
all_truth = all_tests['Truth'] == 'leak'
accuracy_leak = ((all_tests['Direction'] == 'Leakage') & all_truth).sum() / float(all_truth.sum())

all_truth = all_tests['Truth'] == 'independent'
accuracy_independent = ((all_tests['Direction'] == 'Independent') & all_truth).sum() / float(all_truth.sum())

all_truth = (all_tests['Truth'] == 'cause')
accuracy_cause = ((all_tests['Direction'] == 'No Leakage') & all_truth).sum() / float(all_truth.sum())

In [180]:
accuracy_leak, accuracy_independent, accuracy_cause

(0.33333333333333331, 1.0, 0.0)

# Since we know the groun truth, we can test that the errors are independent of the causing variable. This would tell us if the problem in estimation is with the regression or the independence test

In [61]:
hsic = it.HSIC_b((Y['values'] - all_features['leak_0']['values']).astype('float'), Y['values'].astype('float'))

In [62]:
hsic.empirical_test()

2.7476125829033291e-11

In [63]:
hsic = it.HSIC_b((Y['values'] - all_features['X_0']['values']).astype('float'), Y['values'].astype('float'))

In [64]:
hsic.empirical_test()

0.035625346331722825